# 03 - Streaming Silver & Silver_ML

Pipeline de transformation :
- **Bronze → Silver** : Nettoyage et enrichissement
- **Bronze → Silver_ML** : Feature engineering pour le Machine Learning

## Configuration

In [5]:
from pyspark.sql.functions import (
    col, from_unixtime, to_timestamp, round,
    lag, avg, stddev, row_number, when, sqrt, pow, lit, min as spark_min, broadcast
)
from pyspark.sql.window import Window
from config import get_s3_path, create_spark_session

BRONZE_PATH = get_s3_path("bronze", "flights")
SILVER_PATH = get_s3_path("silver", "flights")
SILVER_ML_PATH = get_s3_path("silver", "flights_ml")
CHECKPOINT_SILVER = get_s3_path("checkpoints", "silver_flights")
CHECKPOINT_SILVER_ML = get_s3_path("checkpoints", "silver_ml_flights")
AIRPORTS_CSV = "./data/airports.csv"

spark = create_spark_session("StreamingSilver")

print(f"✅ Input:     {BRONZE_PATH}")
print(f"✅ Silver:    {SILVER_PATH}")
print(f"✅ Silver_ML: {SILVER_ML_PATH}")

✅ Spark Session 'StreamingSilver' configurée
✅ Input:     s3a://datalake/bronze/flights
✅ Silver:    s3a://datalake/silver/flights
✅ Silver_ML: s3a://datalake/silver/flights_ml


## Chargement des données aéroports (pour Silver_ML)

In [6]:
df_airports = spark.read.option("header", "true").csv(AIRPORTS_CSV).select(
    col("ident").alias("airport_icao"),
    col("name").alias("airport_name"),
    col("iso_country").alias("airport_country"),
    col("latitude_deg").cast("double").alias("airport_lat"),
    col("longitude_deg").cast("double").alias("airport_lon")
).filter(col("type").isin("large_airport", "medium_airport"))

print(f"✅ {df_airports.count()} aéroports chargés")

✅ 5211 aéroports chargés


## Stream 1 : Bronze → Silver

In [7]:
df_bronze_stream = spark.readStream.format("delta").load(BRONZE_PATH)

df_silver = df_bronze_stream \
    .filter(col("icao24").isNotNull()) \
    .filter(col("latitude").isNotNull() & col("longitude").isNotNull()) \
    .withColumn("event_timestamp", to_timestamp(from_unixtime(col("time")))) \
    .withColumn("velocity_kmh", round(col("velocity") * 3.6, 2)) \
    .withColumn("altitude_meters", col("baro_altitude")) \
    .select(
        "event_timestamp", "icao24", "callsign", "origin_country",
        "longitude", "latitude", "velocity_kmh", "altitude_meters",
        "on_ground", "category"
    )

print(f"🚀 Stream 1: Bronze → Silver")

query_silver = df_silver.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", CHECKPOINT_SILVER) \
    .option("mergeSchema", "true") \
    .start(SILVER_PATH)

🚀 Stream 1: Bronze → Silver


26/01/23 15:38:16 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
26/01/23 15:38:16 WARN StreamingQueryManager: Stopping existing streaming query [id=03cb7525-313b-4658-807f-c1584735ae56, runId=1af062d8-10ee-4697-b766-c1f13030c8bb], as a new run is being started.
26/01/23 15:38:16 ERROR TorrentBroadcast: Store broadcast broadcast_608 fail, remove all pieces of the broadcast


## Stream 2 : Bronze → Silver_ML (Feature Engineering)

Transformation avec features pour le ML, directement depuis Bronze.

In [8]:
def process_ml_batch(batch_df, batch_id):
    """Traitement d'un micro-batch pour Silver_ML avec feature engineering."""
    
    if batch_df.isEmpty():
        return
    
    # Transformation Bronze → format Silver
    df_base = batch_df \
        .filter(col("icao24").isNotNull()) \
        .filter(col("latitude").isNotNull() & col("longitude").isNotNull()) \
        .withColumn("event_timestamp", to_timestamp(from_unixtime(col("time")))) \
        .withColumn("velocity_kmh", round(col("velocity") * 3.6, 2)) \
        .withColumn("altitude_meters", col("baro_altitude"))
    
    # Nettoyage ML
    df_clean = df_base \
        .filter(col("altitude_meters").between(-500, 15000)) \
        .filter(col("velocity_kmh").between(0, 1200))
    
    if df_clean.isEmpty():
        return
    
    # Features temporelles
    window_aircraft = Window.partitionBy("icao24").orderBy("event_timestamp")
    
    df_temporal = df_clean \
        .withColumn("prev_altitude", lag("altitude_meters", 1).over(window_aircraft)) \
        .withColumn("prev_velocity", lag("velocity_kmh", 1).over(window_aircraft)) \
        .withColumn("altitude_change", col("altitude_meters") - col("prev_altitude")) \
        .withColumn("velocity_change", col("velocity_kmh") - col("prev_velocity")) \
        .withColumn("observation_rank", row_number().over(window_aircraft))
    
    # Jointure aéroports
    df_on_ground = df_temporal.filter(col("on_ground") == True)
    df_in_flight = df_temporal.filter(col("on_ground") == False)
    
    if df_on_ground.count() > 0:
        df_with_airports = df_on_ground.crossJoin(broadcast(df_airports)).withColumn(
            "dist", sqrt(pow(col("latitude") - col("airport_lat"), 2) + pow(col("longitude") - col("airport_lon"), 2))
        )
        
        w = Window.partitionBy("icao24", "event_timestamp")
        df_closest = df_with_airports.withColumn("min_dist", spark_min("dist").over(w)) \
            .filter(col("dist") == col("min_dist")) \
            .drop("dist", "min_dist", "airport_lat", "airport_lon")
        
        df_enriched = df_closest.unionByName(
            df_in_flight.withColumn("airport_icao", lit(None))
                        .withColumn("airport_name", lit(None))
                        .withColumn("airport_country", lit(None)),
            allowMissingColumns=True
        )
    else:
        df_enriched = df_in_flight \
            .withColumn("airport_icao", lit(None)) \
            .withColumn("airport_name", lit(None)) \
            .withColumn("airport_country", lit(None))
    
    # Features rolling window
    rolling_window = Window.partitionBy("icao24").orderBy("event_timestamp").rowsBetween(-5, 0)
    
    df_rolling = df_enriched \
        .withColumn("rolling_avg_altitude", avg("altitude_meters").over(rolling_window)) \
        .withColumn("rolling_std_altitude", stddev("altitude_meters").over(rolling_window)) \
        .withColumn("rolling_avg_velocity", avg("velocity_kmh").over(rolling_window))
    
    # Label flight_phase
    df_ml = df_rolling.withColumn(
        "flight_phase",
        when(col("on_ground") == True, "GROUND")
        .when((col("altitude_change") > 50) & (col("altitude_meters") < 3000), "TAKEOFF")
        .when(col("altitude_change") > 20, "CLIMB")
        .when(col("altitude_change").between(-20, 20) & (col("altitude_meters") > 8000), "CRUISE")
        .when(col("altitude_change") < -20, "DESCENT")
        .otherwise("TRANSITION")
    )
    
    # Sélection des colonnes finales
    df_final = df_ml.select(
        "event_timestamp", "icao24", "callsign", "origin_country",
        "longitude", "latitude", "velocity_kmh", "altitude_meters",
        "on_ground", "category",
        "prev_altitude", "prev_velocity", "altitude_change", "velocity_change", "observation_rank",
        "airport_icao", "airport_name", "airport_country",
        "rolling_avg_altitude", "rolling_std_altitude", "rolling_avg_velocity",
        "flight_phase"
    )
    
    # Écriture
    df_final.write.format("delta").mode("append").save(SILVER_ML_PATH)

In [9]:
df_bronze_ml_stream = spark.readStream.format("delta").load(BRONZE_PATH)

print(f"🚀 Stream 2: Bronze → Silver_ML (Feature Engineering)")

query_silver_ml = df_bronze_ml_stream.writeStream \
    .foreachBatch(process_ml_batch) \
    .option("checkpointLocation", CHECKPOINT_SILVER_ML) \
    .start()

🚀 Stream 2: Bronze → Silver_ML (Feature Engineering)


26/01/23 15:38:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


## Monitoring des streams

In [ ]:
import time

print("📊 Monitoring des streams (Ctrl+C pour arrêter)")
print("="*60)

try:
    while True:
        print(f"\n⏱️  {time.strftime('%H:%M:%S')}")
        print(f"  Silver:    {query_silver.status}")
        print(f"  Silver_ML: {query_silver_ml.status}")
        time.sleep(30)
except KeyboardInterrupt:
    print("\n⏹️  Arrêt demandé...")

📊 Monitoring des streams (Ctrl+C pour arrêter)

⏱️  15:38:32
  Silver:    {'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}
  Silver_ML: {'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}


26/01/23 15:38:39 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
26/01/23 15:38:57 ERROR NonFateSharingFuture: Failed to get result from future  
scala.runtime.NonLocalReturnControl
26/01/23 15:39:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers



⏱️  15:39:02
  Silver:    {'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}
  Silver_ML: {'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}


26/01/23 15:39:15 ERROR NonFateSharingFuture: Failed to get result from future  
scala.runtime.NonLocalReturnControl
26/01/23 15:39:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers



⏱️  15:39:32
  Silver:    {'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}
  Silver_ML: {'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}


...


⏱️  15:52:06


  Silver:    {'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}
  Silver_ML: {'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}


26/01/23 15:52:21 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
26/01/23 15:52:22 ERROR NonFateSharingFuture: Failed to get result from future  
scala.runtime.NonLocalReturnControl



⏱️  15:52:36
  Silver:    {'message': 'Waiting for data to arrive', 'isDataAvailable': False, 'isTriggerActive': False}
  Silver_ML: {'message': 'Processing new data', 'isDataAvailable': True, 'isTriggerActive': True}



⏱️  15:53:06
  Silver:    {'message': 'Getting offsets from DeltaSource[s3a://datalake/bronze/flights]', 'isDataAvailable': False, 'isTriggerActive': True}
  Silver_ML: {'message': 'Getting offsets from DeltaSource[s3a://datalake/bronze/flights]', 'isDataAvailable': False, 'isTriggerActive': True}

⏱️  15:53:36
  Silver:    {'message': 'Getting offsets from DeltaSource[s3a://datalake/bronze/flights]', 'isDataAvailable': False, 'isTriggerActive': True}
  Silver_ML: {'message': 'Getting offsets from DeltaSource[s3a://datalake/bronze/flights]', 'isDataAvailable': False, 'isTriggerActive': True}

⏱️  15:54:06
  Silver:    {'message': 'Getting offsets from DeltaSource[s3a://datalake/bronze/flights]', 'isDataAvailable': False, 'isTriggerActive': True}
  Silver_ML: {'message': 'Getting offsets from DeltaSource[s3a://datalake/bronze/flights]', 'isDataAvailable': False, 'isTriggerActive': True}

⏱️  15:54:36
  Silver:    {'message': 'Waiting for data to arrive', 'isDataAvailable': False, 'isTr

## Arrêt des streams

In [ ]:
query_silver.stop()
query_silver_ml.stop()
print("✅ Tous les streams arrêtés")

✅ Tous les streams arrêtés


## Vérification

In [8]:
print("📊 Statistiques :")
print(f"  Bronze:    {spark.read.format('delta').load(BRONZE_PATH).count():,} lignes")
print(f"  Silver:    {spark.read.format('delta').load(SILVER_PATH).count():,} lignes")
print(f"  Silver_ML: {spark.read.format('delta').load(SILVER_ML_PATH).count():,} lignes")

print("\n📊 Distribution flight_phase (Silver_ML) :")
spark.read.format("delta").load(SILVER_ML_PATH).groupBy("flight_phase").count().orderBy("count", ascending=False).show()

📊 Statistiques :
  Bronze:    98,840 lignes
  Silver:    97,950 lignes
  Silver_ML: 88,432 lignes

📊 Distribution flight_phase (Silver_ML) :
+------------+-----+
|flight_phase|count|
+------------+-----+
|      CRUISE|37190|
|  TRANSITION|29374|
|     DESCENT|10699|
|       CLIMB| 8877|
|     TAKEOFF| 2035|
|      GROUND|  257|
+------------+-----+

